In [53]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pandas as pd 

## Attempt 1: Remove more features

## Preprocessing

In [54]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [55]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME', 'CLASSIFICATION', 'AFFILIATION', 'STATUS', 'SPECIAL_CONSIDERATIONS'], axis="columns")

In [56]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [57]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [58]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [59]:
# Convert categorical data to numeric with `pd.get_dummies`
application_processed = pd.get_dummies(application_df)

application_processed

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,USE_CASE_CommunityServ,USE_CASE_Heathcare,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,True,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,True,False,False,False,False,True,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34295,5000,0,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34296,5000,0,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34297,5000,1,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [60]:
# Split our preprocessed data into our features and target arrays
X = application_processed.drop('IS_SUCCESSFUL', axis = "columns")
y = application_processed['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [61]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [62]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 80)                2080      
                                                                 
 dense_27 (Dense)            (None, 30)                2430      
                                                                 
 dense_28 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4541 (17.74 KB)
Trainable params: 4541 (17.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [63]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [64]:
# Train the model
fit_model = nn1.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6494 - accuracy: 0.6243
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6432 - accuracy: 0.6313
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6424 - accuracy: 0.6331
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6418 - accuracy: 0.6333
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6413 - accuracy: 0.6335
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6410 - accuracy: 0.6341
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6407 - accuracy: 0.6338
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6404 - accuracy: 0.6336
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6405 - accuracy: 0.6342
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6401 - accura

In [65]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6398 - accuracy: 0.6338 - 477ms/epoch - 2ms/step
Loss: 0.6398087739944458, Accuracy: 0.6338192224502563


In [66]:
# Export our model to HDF5 file
nn1.save('AlphabetSoupCharity_Optimization_1.h5')

c:\Users\tamhl\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Attempt 2

Back to oringal scaled data. Increase epochs

# Preprocessing

In [67]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis="columns")

# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
    
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
    
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts<400].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Convert categorical data to numeric with `pd.get_dummies`
application_processed = pd.get_dummies(application_df)

In [68]:
# Split our preprocessed data into our features and target arrays
X = application_processed.drop('IS_SUCCESSFUL', axis = "columns")
y = application_processed['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [69]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 80)                3360      
                                                                 
 dense_30 (Dense)            (None, 30)                2430      
                                                                 
 dense_31 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5821 (22.74 KB)
Trainable params: 5821 (22.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [70]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model (Increased epochs to 200)
fit_model = nn2.fit(X_train_scaled,y_train,epochs=300)

# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn2.save('AlphabetSoupCharity_Optimization_2.h5')

Epoch 1/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5736 - accuracy: 0.7203
Epoch 2/300
804/804 [==============================] - 1s 2ms/step - loss: 0.5582 - accuracy: 0.7306
Epoch 3/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5558 - accuracy: 0.7309
Epoch 4/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5535 - accuracy: 0.7319
Epoch 5/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7318
Epoch 6/300
804/804 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7320
Epoch 7/300
804/804 [==============================] - 1s 2ms/step - loss: 0.5500 - accuracy: 0.7334
Epoch 8/300
804/804 [==============================] - 1s 2ms/step - loss: 0.5499 - accuracy: 0.7348
Epoch 9/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7346
Epoch 10/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accura

c:\Users\tamhl\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Attempt 3

Increase number of hidden layers

In [71]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  60
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 20

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 60)                2520      
                                                                 
 dense_33 (Dense)            (None, 20)                1220      
                                                                 
 dense_34 (Dense)            (None, 20)                420       
                                                                 
 dense_35 (Dense)            (None, 1)                 21        
                                                                 
Total params: 4181 (16.33 KB)
Trainable params: 4181 (16.33 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [72]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model (Increased epochs to 200)
fit_model = nn3.fit(X_train_scaled,y_train,epochs=300)

# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn3.save('AlphabetSoupCharity_Optimization_3.h5')

Epoch 1/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5752 - accuracy: 0.7187
Epoch 2/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7304
Epoch 3/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7318
Epoch 4/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7324
Epoch 5/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7333
Epoch 6/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7331
Epoch 7/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7341
Epoch 8/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7342
Epoch 9/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7345
Epoch 10/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accura